In [2]:
import os
import sys
import subprocess
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.DELAT_utils import match_h5_files_by_channels, setup_logging, collect_region_stats_paths
from src.registration import register_and_transform
from src.stats import compute_region_stats
import logging

In [3]:
animal = 'ANM550749_left_JF552'
base_dir = r'X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains'

# Set the parameters manually here

fx_path =  r'X:\Boaz\I2\atlas10_hemi.tif'
param_files_dir = r'X:\Boaz\I2\itk'
annotation_path =  r'X:\Boaz\I2\annotation_10_hemi.nii'

In [4]:
logger = setup_logging(base_dir, animal)
# Load the fixed image (fx)
logger.info(f"Loading fixed image (fx) from {fx_path}.")
fx = itk.imread(fx_path, pixel_type=itk.US)

# Load the parameter files
param_files = [
    os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
    os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
]
logger.info(f"Loaded parameter files from {param_files_dir}.")



2024-11-13 21:29:00 - INFO - Logging initialized for animal processing.
2024-11-13 21:29:00 - INFO - Loading fixed image (fx) from X:\Boaz\I2\atlas10_hemi.tif.
2024-11-13 21:29:17 - INFO - Loaded parameter files from X:\Boaz\I2\itk.


In [5]:
# Load the annotation volume
logger.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


2024-11-13 21:29:17 - INFO - Loading annotation volume from X:\Boaz\I2\annotation_10_hemi.nii.


(dtype('uint64'), (800, 1320, 658))

In [6]:
# Match H5 files by channels for all animals
logger.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)
animals_files

2024-11-13 21:30:10 - INFO - Matching H5 files in X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains.


{'ANM549057_left_JF552': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM549057_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM550749_left_JF552': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM550751_left_JF673': {'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550751_lef

In [7]:
files = animals_files.get(animal)
files

{'ch0': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch1': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch2': 'X:\\Boaz\\I2\\2024-09-19_iDISCO_CalibrationBrains\\ANM550749_left_JF552\\uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'}

In [8]:
output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
logger.info(f"Using output dir: {output_dir}")

2024-11-13 21:30:10 - INFO - Using output dir: X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk


In [14]:
def invert_affine_transform(parameter_file):
    # Load the affine transformation
    transform = itk.ParameterObject.New()
    transform.AddParameterFile(parameter_file)
    matrix = np.array(transform.GetParameter(0, 'TransformParameters'))[:9].astype(float).reshape(3, 3)
    translation = np.array(transform.GetParameter(0, 'TransformParameters')[9:12]).astype(float)

    # Invert the matrix
    inv_matrix = np.linalg.inv(matrix)
    inv_translation = -inv_matrix @ translation

    # Create an inverse transformation file
    inv_transform_params = list(map(str, inv_matrix.flatten())) + list(map(str, inv_translation))
    
    # Create a new ParameterObject with the inverted transformation
    inverted_transform = itk.ParameterObject.New()
    inverted_transform.AddParameterFile(parameter_file)
    inverted_transform.SetParameter(0, 'TransformParameters', inv_transform_params)
    return inverted_transform

In [15]:
affine = os.path.join(output_dir, 'TransformParameters.0.txt')
inverse_affine = invert_affine_transform(affine)

In [ ]:
affine

In [18]:
inverse_affine

<itk.elxParameterObjectPython.elastixParameterObject; proxy of <Swig Object of type 'elastixParameterObject *' at 0x000001A1297AE6D0> >

In [19]:
num_cores = os.cpu_count()  
itk.MultiThreaderBase.SetGlobalDefaultNumberOfThreads(48)
logger.info(f"Register_and_transform started, set ITK to use {num_cores/2} threads.")

2024-11-13 21:34:12 - INFO - Register_and_transform started, set ITK to use 48.0 threads.


In [20]:

from DELAT_utils import read_h5_image

In [21]:
mv = read_h5_image(files['ch0'], 'Data')

In [ ]:
inverse_affine

In [23]:
inverse_affine.WriteParameterFile(inverse_affine.GetParameterMap(0), 
                                   os.path.join(output_dir, 'inverse_affine.txt'))

In [25]:
init_affine = os.path.join(output_dir, 'inverse_affine.txt')

In [45]:
parameter_object = itk.ParameterObject.New()
parameter_object.AddParameterFile(init_affine)
# Set up Transformix filter to apply the transformation
transformix_filter = itk.TransformixFilter.New(Input=fx, TransformParameterObject=parameter_object)
transformix_filter.SetComputeSpatialJacobian(False)
transformix_filter.SetComputeDeterminantOfSpatialJacobian(False)
transformix_filter.SetComputeDeformationField(False)
transformix_filter.Update()

# Get the transformed image
transformed_image = transformix_filter.GetOutput()

# Save the transformed image
itk.imwrite(transformed_image,  os.path.join(output_dir,'test_atlas.tif'))

In [31]:
def update_transformix_parameters(transform_file, target_image):
    # Load parameters from the transform file
    with open(transform_file, 'r') as file:
        lines = file.readlines()
    
    # Get target image properties
    size = " ".join(map(str, itk.size(target_image)))
    spacing = " ".join(map(str, itk.spacing(target_image)))
    origin = " ".join(map(str, itk.origin(target_image)))
    
    # Update lines with the size, spacing, and origin from the target image
    updated_lines = []
    for line in lines:
        if line.startswith("(Size "):
            updated_lines.append(f"(Size {size})\n")
        elif line.startswith("(Spacing "):
            updated_lines.append(f"(Spacing {spacing})\n")
        elif line.startswith("(Origin "):
            updated_lines.append(f"(Origin {origin})\n")
        else:
            updated_lines.append(line)
    
    # Save the modified parameter file
    with open(transform_file, 'w') as file:
        file.writelines(updated_lines)
    print(f"Updated transform file saved to {transform_file}")

In [42]:
fx.GetSize()

AttributeError: 'itkImageUS3' object has no attribute 'GetSize'

In [43]:
update_transformix_parameters(init_affine, mv_itk)

Updated transform file saved to X:\Boaz\I2\2024-09-19_iDISCO_CalibrationBrains\ANM550749_left_JF552\itk\inverse_affine.txt


In [22]:
parameter_object = itk.ParameterObject.New()
parameter_object.AddParameterFile(inverse_affine)  # Add initial transformation first
for p in param_files:
    parameter_object.AddParameterFile(p)  # Add the rest of the registration parameters

logger.info("Initial and parameter files read")

logger.info("param files read")
# Create output directory if it doesn't exist
if not os.path.isdir(output_dir):
    os.makedirs(output_dir, exist_ok=True)
    logger.info(f"Created output directory {output_dir}")



TypeError: in method 'elastixParameterObject_AddParameterFile', argument 2 of type 'std::string const &'

In [29]:
reference_image =  itk.image_from_array(mv)
reference_image

<itk.itkImagePython.itkImageUS3; proxy of <Swig Object of type 'itkImageUS3 *' at 0x00000290C7F7EE80> >

In [ ]:
reference_image.SetSpacing([10.0, 10.0, 10.0])  # Replace with actual spacing if known
reference_image.SetOrigin([0.0, 0.0, 0.0])
reference_image.SetDirection(itk.Matrix[itk.D, 3, 3]())  # Identity matrix for direction

In [ ]:
itk

In [48]:
transformix_filter = itk.TransformixFilter[itk.Image[itk.F, 3]].New()
transformix_filter.SetTransformParameterObject(params)
transformix_filter.SetMovingImage(reference_image)
transformix_filter.SetOutputDirectory(output_dir)
transformix_filter.SetLogToConsole(True)
transformix_filter.UpdateLargestPossibleRegion()
transformix_filter.SetOutputDirectory(output_dir)
transformix_filter.Update()

RuntimeError: D:\a\im\build\cp311-abi3-win_amd64\_deps\elx-src\Core\Main\itkTransformixFilter.hxx:379:
ITK ERROR: No entry Spacing found in transformParameterMap

In [41]:
itk.TransformixFilter.GetTypes()

<itkTemplate itk::TransformixFilter>
Options:
  [<class 'itk.itkImagePython.itkImageD2'>,]
  [<class 'itk.itkImagePython.itkImageD3'>,]
  [<class 'itk.itkImagePython.itkImageD4'>,]
  [<class 'itk.itkImagePython.itkImageF2'>,]
  [<class 'itk.itkImagePython.itkImageF3'>,]
  [<class 'itk.itkImagePython.itkImageF4'>,]
  [<class 'itk.itkImagePython.itkImageSS2'>,]
  [<class 'itk.itkImagePython.itkImageSS3'>,]
  [<class 'itk.itkImagePython.itkImageSS4'>,]
  [<class 'itk.itkImagePython.itkImageUC2'>,]
  [<class 'itk.itkImagePython.itkImageUC3'>,]
  [<class 'itk.itkImagePython.itkImageUC4'>,]
  [<class 'itk.itkImagePython.itkImageUS2'>,]
  [<class 'itk.itkImagePython.itkImageUS3'>,]
  [<class 'itk.itkImagePython.itkImageUS4'>,]


In [42]:
transformix_filter

<itk.itkTransformixFilterPython.itkTransformixFilterISS2; proxy of <Swig Object of type 'itkTransformixFilterISS2 *' at 0x00000290C86B2430> >